# Sentiment Analysis of Bengali Texts
We will go through all the steps towards Sentiment Analaysis of Bengali Language with Deep Learning Model. 

There are very few published works on this topic. Specially when I was working with Sentiment Analysis of Bengali Language for my final year's thesis I didn't find any stuctured work that will lead me from scratch to results. So I am writting the code bellow that will lead from loading dataset to constructing model, to make prediction with the model.

Withot further discussion, let's dive into code. I will be writing the code in google colab. So if you run in your local environment make sure you have installed all the packages/modules. And All my files will be uploaded from drive. You can try it with your own dataset if you want.

First, let's Check if the colab GPU is connected, execute following code

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

UsageError: Line magic function `%tensorflow_version` not found.


Let's import neessary libraries

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D,Conv1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer



seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

2022-01-13 03:02:15.801615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 03:02:15.801644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# BNLP Toolkit

Before feeding data to a Neural Network, we need to prepare the data for the neural network. To remove noise from the data, and to make it fit for the neural network, we will go throuth several steps 

In the first steps of prepocessing, BNLP tool kit will help us. We will use it for POS tagging, punctuation removal.

In [6]:
!pip install bnlp_toolkit


Defaulting to user installation because normal site-packages is not writeable


In [7]:
print(tf.keras.__version__)
print(tf.__version__)
!python --version

2.7.0
2.7.0
Python 2.7.18


# Dataset
The following blocks will load the dataset and we'll see their shapes.

I have used "Bengali Book Review Dataset" which was published by E. Hossain and his team. Thanks to them. Go google for it and you will find the dataset. I am not providing the original dataset here for some reason.


N.B.: I am loading data from my google drive. Make sure you give appropriate path to your dataset

In [8]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
movie_reviews = pd.read_excel(r"/content/drive/My Drive/Thesis/bookReviews_2000.xlsx")

movie_reviews.isnull().values.any()

movie_reviews.shape

In [ ]:
movie_reviews.head()

In [ ]:
#movie_reviews["Text"][3]

In [ ]:
import seaborn as sns

sns_plot=sns.countplot(x='Sentiment', data=movie_reviews)
fig = sns_plot.get_figure()
fig.savefig("data_shape.png")

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)
    
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [2]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

NameError: name 're' is not defined

In [ ]:
Z=[]
Sent=[]
sentence_tokens_list = []
word_tokens_list = []
pos_tags_list = []
sentences = []

sentence_tokens_list.clear()
word_tokens_list.clear()
pos_tags_list.clear()
sentences.clear()


sentence = list(movie_reviews['Review'])

for sen in sentence:
    sentences.append(preprocess_text(sen))


In [ ]:
print(sentences[:5])

In [ ]:
# X.clear()

Z.clear()

Sent.clear()


# POS tagging and Punctuation Removal

In the following code, we will first have word tokens from each sentence with the help of BNLP toolkit. Then with POS tagging, we'll detect punctuations and remove them. 

In [1]:
import nltk
nltk.download('punkt')
# X.clear()
Z.clear()
local_pos_tags = []
local_pos_tags.clear()
pos_tags_list.clear()

import copy ## to implement shallow copy. See copy in python

from bnlp import NLTKTokenizer
from bnlp import POS
bn_pos = POS()
model_path = "/content/drive/My Drive/Thesis/bn_pos_model.pkl"
for i in range(len(sentences)):
    text = sentences[i]
    bnltk = NLTKTokenizer()
    
    
    word_tokens = bnltk.word_tokenize(text)
    word_tokens_copy = copy.copy(word_tokens)
#     word_tokens_list.append(word_tokens)
    
    
    sentence_tokens = bnltk.sentence_tokenize(text)
    sentence_tokens_list.append(sentence_tokens)
    
    for j in range(len(word_tokens)):
        word = word_tokens[j]    
        pos_tags = bn_pos.tag(model_path, word)
        
        if(pos_tags[0][1] == "PU"):
            word_tokens_copy.remove(str(word))

            
    word_tokens_list.append(word_tokens_copy)

            

print("\n Sentence Tokens")
print(sentence_tokens_list[:3])

print("\n Word Tokens")
print(word_tokens_list[:3])




[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


NameError: name 'Z' is not defined

In [ ]:
## We are creating word tokens first in 'word_tokens' . Then a shallow copy is made of 'word_tokens' to 
## 'word_tokens_copy'. We then check the POS tag of the tokens and if any one of them is "PU" (i.e punctuation)
## then we remove it from 'word_tokens_copy'. After iterating through all the tokens (hence removing all the "PU")
## we add 'word_tokens_copy' to 'word_tokens_list' .

## All these mess is done to remove the error 'list index out of range' while removing an element

In [ ]:
import copy
copy_of_word_tokens_list = [] ## It is costly to compute "word_tokens_list".
                            ## So I'm keeping an copy so that I can experiment with one and change it as wish
copy_of_word_tokens_list.clear()
copy_of_word_tokens_list = copy.copy(word_tokens_list)

In [ ]:
word_tokens_list = copy.copy(copy_of_word_tokens_list)

While removing punctations, there were some english sentence and they were removed. So, with the following block, I'm removing those empty arrays. In this way, we have no english sentance in the dataset. The dataset consists of only the Bangla Sentences.

In [ ]:


remove_counter = 0  ## to check if any array is removed for being empty
token_lngth = 0
token_length = len(word_tokens_list) ## to iterate over the whole "word_tokens_list" array
empty_positions = [] ## keep track of empty positions
empty_positions.clear()
print(token_length) ## How many positions are empty actually 

for i in range(token_length):
    if(len(copy_of_word_tokens_list[i]) == 0):
        word_tokens_list.pop(i)
        empty_positions.append(i)
        remove_counter += 1
        print("a")
        
        
print(remove_counter)

In [ ]:
print(len(word_tokens_list))

print(len(sentence_tokens_list))


# Word2vec

Now we will generate our own wordvector model from all the words in our dataset which will be used later to generate embeddding matrix. If are new and don't know about what wordvector and embedding matrix are, please go google it, you will have some great help.


To generate wordvector model, we'll use gensim word2vec with skip-gram model.

In [ ]:
#word2vec
import json
import os
import re
import string
import numpy as np

from gensim.models import Word2Vec


In [ ]:
model1 =Word2Vec(word_tokens_list, vector_size=100, window=15, min_count=1,sg=0)
#model2 =Word2Vec.load('bn_w2v_model.text')
model1.wv.save_word2vec_format("/content/drive/My Drive/Thesis/myword2vec_model.txt")

Let's see an example how words are stored in wordvector

In [ ]:
model1.wv["ভালো"]


In [ ]:
y = movie_reviews['Sentiment']

print(len(y))


In the following block, we'll remove the label of corresponding english sentances from dataset, as we have removed them previously.

In [ ]:

for i in range(len(empty_positions)):
    y.pop(empty_positions[i])

print(len(y))


# Train-Test Split

now we'll devide the total data into train and test set. I have taken 10% data for test. Feel free to chose the percentage as you choice

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(word_tokens_list, y, test_size=0.10, random_state=42)

print(X_train[:5])
print(y_train[:5])

In [ ]:
print(len(X_train))

len(X_test)

# Tokenization

Neural networks can only work with numbers. But all we have are texts. So to feed them into neural-net, we must convert them to numbers. With Tokenizzation we have done it in the following block. Again, if you don't know what tokenization is, go search for it. 

In [ ]:

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

Let's see the result of tokenization with an example

In [ ]:

X_train[:5]

# Padding

Padding is necessary as input to a neural network is always fixed sized. But the sentences we are working on are of different lengths. With padding, we add extra zeros to make them of the same length. 
I have used post padding and set pad length to 100.  

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

# print(vocab_size)

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
X_train[:5]

# Embedding matrix

Now we'll generate our embedding matrix. We'll use our previously generated wordvector model to generate the embedding matrix which will later be used in the embedding layer of our Neural Network.

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/drive/My Drive/Thesis/myword2vec_model.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')

    embeddings_dictionary [word] = vector_dimensions
glove_file.close()


embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():

    embedding_vector = embeddings_dictionary.get(word)

    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector


In [ ]:
embedding_matrix[10]


# Neural Network / Deep Learning model.\

All of our prerpocessing tasks are done! Now we'll build our deep learning model which will perform sentiment analysis for us. 

In my model, I have used a combination of CNN and RNN Layers with the Dense layers. 

I had done some hyper-parameter tuning for my model. Feel free to use your won model/layers/parameter as you want. This part is totally on you.

In [ ]:
from keras import Sequential, optimizers
from keras.layers import Embedding, LSTM, Dense, Dropout,Bidirectional,ConvLSTM2D,GRU,SimpleRNN,RNN
from keras import regularizers

model2 = Sequential()
embedding_layer = Embedding(vocab_size, 100,weights=[embedding_matrix], input_length=maxlen , trainable=True)
model2.add(embedding_layer)

model2.add (Bidirectional(LSTM(64,  return_sequences=True,kernel_regularizer=regularizers.l2(0.15))))

model2.add(Conv1D(32, 4, activation='relu'))
model2.add (LSTM(32,return_sequences=True, activation = 'tanh') )

model2.add(Conv1D(16, 4, activation='relu'))

model2.add(GlobalMaxPooling1D())

model2.add (Dropout(0.4))

model2.add(Dense(16,activation='relu'))


model2.add(Dense(1, activation='sigmoid'))


opt = optimizers.Adam(learning_rate=0.0001)
#lr=0.0005
model2.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])



Let's see the model's summary

In [ ]:
model2.summary()

# Train the model

And train the model with our training data. Again, feel free to chose the batch_size, epochs etc as you wish.

In [ ]:
history = model2.fit(X_train, y_train, batch_size=32, epochs=70, verbose=1, validation_split=0.10)

score = model2.evaluate(X_test, y_test, verbose=1)

# Plot the model's performance

The following block will generate model's accuracy and loss graph with training and validation data.

In [3]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc = 'upper left')
plt.savefig('/content/drive/My Drive/Thesis/model_Accuracy.png',dpi=600)

plt.show()


plt.figure(figsize=(10, 8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc = 'upper left')
plt.savefig('/content/drive/My Drive/Thesis/model_loss.png', dpi=600)

plt.show()

NameError: name 'history' is not defined

<Figure size 720x576 with 0 Axes>

# Accuracy measures 

Now we will test our model with the test data that our model has never seen. 

We will check the accuracy, recall, precision and f-1 score.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score



yhat_classes = model2.predict_classes(X_test, verbose=1)
yhat_classes = yhat_classes[:, 0]
accuracy = accuracy_score(y_test, yhat_classes)

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)



# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes)
print('Precision: %f' % precision)


# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes)
print('Recall: %f' % recall)



# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes)
print('F1 score: %f' % f1)


# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)



# Confusion Matrix

In [ ]:
# confusion_mtx = tf.math.confusion_matrix(y_true, y_pred) 
plt.figure(figsize=(10, 8))
sns.heatmap(matrix, xticklabels='01', yticklabels='01', annot=True, fmt='g')


plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig('/content/drive/My Drive/Thesis/confussion_matrix.png', dpi=600)

plt.show()

# ROC curve

In [ ]:
fpr, tpr, _ = roc_curve(y_test,yhat_classes)
# plt.figure(figsize=(14,8))
plt.plot(fpr, tpr, color="red")
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Roc curve')
plt.savefig('/content/drive/My Drive/Thesis/roc_curve.png', dpi=600)

plt.show()